In [1]:
import os 
import tensorflow as tf

In [2]:
root_dir = "../"
root_dir = os.path.abspath(root_dir)
data_path = os.path.join(root_dir, "data")
processed_path = os.path.join(data_path, "processed")

In [3]:
file_list =[os.path.join(processed_path, x) for x in os.listdir(processed_path)]
file_list = file_list[1:]#.gitkeep

In [10]:
def parse_tfrecord_fn(example_proto):
    feature_description = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)
    
    height = tf.cast(example['height'], tf.int32)
    width = tf.cast(example['width'], tf.int32)
    depth = tf.cast(example['depth'], tf.int32)
    label = tf.cast(example['label'], tf.int32)

    image_raw = tf.io.decode_raw(example['image_raw'], tf.int32)
    image = tf.reshape(image_raw, (height, width, depth))
    

    return image, label

In [22]:
tf_set = tf.data.TFRecordDataset(file_list)
tf_set

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [23]:
tf_set = tf_set.map(parse_tfrecord_fn)
tf_set

<_MapDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [24]:
# Create an iterator and get the next element
iterator = iter(tf_set)
for i in range(3):
    image, label = iterator.get_next()
    print(image.shape)

(256, 256, 38)
(256, 256, 38)
(256, 256, 38)


In [14]:
tf_set=tf_set.shuffle(1000).batch(64)
tf_set

<_BatchDataset element_spec=(TensorSpec(shape=(None, None, None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [15]:
import keras

class DenseOutput(keras.layers.Layer):
    def __init__(self, output_classes:int):
        super().__init__()
        self.flatten_layer = keras.layers.Flatten()
        self.dense_1 = keras.layers.Dense(4096, activation='relu')
        self.dense_2 = keras.layers.Dense(4096, activation='relu')
        self.output_layer = keras.layers.Dense(output_classes, activation='softmax')

    def call(self, x):
        x = self.flatten_layer(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.output_layer(x)
        return x

In [16]:
class ConvComponent(keras.layers.Layer):
    def __init__(self, channel_count: int):
        super().__init__()
        self.conv_1 = keras.layers.Conv2D(channel_count, (3, 3), activation='relu', padding='same')
        self.conv_2 = keras.layers.Conv2D(channel_count, (3, 3), activation='relu', padding='same')
        self.pool_layer = keras.layers.MaxPooling2D((2,2), strides=(2,2))

    def call(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.pool_layer(x)
        return x


In [17]:
class Vgg16(keras.Model):
    def __init__(self, output_classes:int):
        super().__init__()
        self.initial_pool = keras.layers.MaxPooling2D((5, 5), strides=(5, 5))
        self.comp_1 = ConvComponent(64)
        self.comp_2 = ConvComponent(128)
        self.comp_3 = ConvComponent(256) #all these have only 2 components (slight deviation)
        self.comp_4 = ConvComponent(512)
        self.comp_5 = ConvComponent(512)
        self.connected_layer = DenseOutput(output_classes)

    def call(self, inputs):
        inputs = self.initial_pool(inputs)
        inputs = self.comp_1(inputs)
        inputs = self.comp_2(inputs)
        inputs = self.comp_3(inputs)
        inputs = self.comp_4(inputs)
        inputs = self.comp_5(inputs)
        inputs = self.connected_layer(inputs)
        return inputs

In [20]:
A = Vgg16(4)
A.compile("adam", keras.losses.SparseCategoricalCrossentropy())

In [25]:
A.fit(tf_set[tf.newaxis, :])

ValueError: Exception encountered when calling Vgg16.call().

[1mInput 0 of layer "max_pooling2d_6" is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: (None, None, None)[0m

Arguments received by Vgg16.call():
  • inputs=tf.Tensor(shape=(None, None, None), dtype=int32)